In [32]:
# !pip install cirq
import cirq
import numpy as np
import random

# **Key Generation Phase**

In [33]:
def quantum_one_way_function(k: str) -> cirq.Circuit:
    """
    Given a classical bit-string k, return a Cirq Circuit that
    prepares the state |f_k> = cos(j*theta)|0> + sin(j*theta)|1>,
    where j = int(k,2) mod L and theta = pi/(2*L) with L = len(k).
    """
    L = len(k)
    j = int(k, 2) % L
    theta = np.pi / (2 * L)

    # Define a single qubit
    q = cirq.LineQubit(0)
    # Create a circuit that applies an Ry rotation (which in Cirq is RY)
    circuit = cirq.Circuit(cirq.ry(2 * j * theta)(q))
    return circuit

# Example usage:
if __name__ == "__main__":
    key = "1011"  # example classical bit-string
    circuit = quantum_one_way_function(key)
    print(circuit)

0: ───Ry(0.75π)───


In [34]:
def prepare_state_circuit(key: str, T: int) -> cirq.Circuit:
    """
    Given a bit-string key, returns a Cirq circuit that prepares T copies of
    the state |f_k> = cos(j*theta)|0> + sin(j*theta)|1> on T distinct qubits.

    This function calls the quantum_one_way_function to generate the circuit for a single qubit,
    and then maps that circuit onto T different qubits.

    Here, L = len(key), j = int(key, 2) % L, and theta = π/(2L).
    """
    circuit = cirq.Circuit()
    qubits = [cirq.LineQubit(i) for i in range(T)]

    for i, q in enumerate(qubits):
        # Get the circuit for a single qubit state prepared by quantum_one_way_function.
        subcircuit = quantum_one_way_function(key)
        # Map the qubit from the subcircuit (which is LineQubit(0)) to our target qubit q.(base is q(0)-->q(i+1))
        mapped_subcircuit = subcircuit.transform_qubits(lambda qubit: q)
        circuit.append(mapped_subcircuit)

    return circuit, qubits


def generate_keys(M: int, L: int, T: int):
    """
    Generates M pairs of random L-bit strings. For each bit-string,
    creates T copies of its corresponding public key state using the quantum one‐way function.

    Returns:
      - private_keys: a list of M tuples (key0, key1) used for signing 0 and 1.
      - public_keys: a dictionary with keys 0 and 1. Each value is a list of M tuples,
        each tuple being (circuit, qubits) representing T copies of |f_k>.
    """
    private_keys = []
    public_keys = {0: [], 1: []}

    for i in range(M):
        # Generate two random bitstrings of length L
        key0 = ''.join(random.choice(['0', '1']) for _ in range(L))
        key1 = ''.join(random.choice(['0', '1']) for _ in range(L))
        private_keys.append((key0, key1))

        # Prepare public key state circuits (with T copies) for each key
        circuit0, qubits0 = prepare_state_circuit(key0, T)
        circuit1, qubits1 = prepare_state_circuit(key1, T)

        public_keys[0].append((circuit0, qubits0))
        public_keys[1].append((circuit1, qubits1))

    return private_keys, public_keys

# Example usage:
if __name__ == "__main__":
    # Set parameters: M = number of key pairs, L = bit-string length, T = copies per key.
    M = 5     # For example, 5 pairs
    L = 4     # Each bit-string is 4 bits long
    T = 3     # Each public key state is given as 3 copies

    private_keys, public_keys = generate_keys(M, L, T)

    print("Private Keys:")
    for idx, (k0, k1) in enumerate(private_keys):
        print(f"Key pair {idx+1}: k0 = {k0}, k1 = {k1}")

    print("\nPublic Keys for message bit 0 (each entry shows the circuit for T copies):")
    for idx, (circuit, qubits) in enumerate(public_keys[0]):
        print(f"\nPublic key state {idx+1}:")
        print(circuit)
        print(qubits)

    print("\nPublic Keys for message bit 1 (each entry shows the circuit for T copies):")
    for idx, (circuit, qubits) in enumerate(public_keys[1]):
        print(f"\nPublic key state {idx+1}:")
        print(circuit)



Private Keys:
Key pair 1: k0 = 1101, k1 = 1111
Key pair 2: k0 = 1101, k1 = 1010
Key pair 3: k0 = 0100, k1 = 0000
Key pair 4: k0 = 0000, k1 = 1011
Key pair 5: k0 = 1110, k1 = 1001

Public Keys for message bit 0 (each entry shows the circuit for T copies):

Public key state 1:
0: ───Ry(0.25π)───────────────────────────

1: ───────────────Ry(0.25π)───────────────

2: ───────────────────────────Ry(0.25π)───
[cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2)]

Public key state 2:
0: ───Ry(0.25π)───────────────────────────

1: ───────────────Ry(0.25π)───────────────

2: ───────────────────────────Ry(0.25π)───
[cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2)]

Public key state 3:
0: ───Ry(0)───────────────────

1: ───────────Ry(0)───────────

2: ───────────────────Ry(0)───
[cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2)]

Public key state 4:
0: ───Ry(0)───────────────────

1: ───────────Ry(0)───────────

2: ───────────────────Ry(0)───
[cirq.LineQubit(0), cirq.LineQubit(

# **Signature Generation & Verification**

In [35]:
def sign_message(b: int, private_keys):
    """
    Given a message bit b (0 or 1) and the list of private key pairs corresponding to b,
    returns the signature: a tuple (b, [k1^b, k2^b, ..., k_M^b]).

    In a real protocol, Alice sends these keys over an insecure classical channel.
    """
    signature_keys = [key_pair[b] for key_pair in private_keys]
    return (b, signature_keys)


In [36]:
def perform_swap_test(state_circuit1, qubit1, state_circuit2, qubit2, repetitions=100):
    """
    Builds and runs a swap-test circuit on two qubits that have been prepared
    by state_circuit1 and state_circuit2 respectively. Returns the fraction of
    runs in which the ancilla measurement is 0.

    The swap test is implemented as:
      - Prepare ancilla in |0>, apply Hadamard.
      - Controlled-SWAP between qubit1 and qubit2 with ancilla as control.
      - Apply Hadamard on ancilla, then measure ancilla.

      H
      If the ancilla is in state |0⟩, nothing happens.
      If the ancilla is in state |1⟩, the two states are swapped.
      H

      Interference:
      After the controlled-SWAP, a second Hadamard is applied to the ancilla.
      The interference created by the two Hadamard gates causes the measurement probability of
      the ancilla to depend on the inner product (overlap) between the two states.


    If the two states are identical, the ancilla is measured as 0 with probability 1.
    """
    # Use a fresh ancilla qubit (choose an index not used by state circuits)
    ancilla = cirq.LineQubit(100)

    # Remap the qubits in the second state-preparation circuit to avoid duplicates.
    # Here we add an offset of 10 to the qubit indices.
    offset = 10
    def remap_qubit(q):
        # q is assumed to be a cirq.LineQubit; create a new one with shifted index.
        return cirq.LineQubit(q.x + offset)

    state_circuit2_remapped = state_circuit2.transform_qubits(remap_qubit)
    # Also update qubit2 accordingly.
    qubit2_remapped = remap_qubit(qubit2)


    # Append the state-preparation circuits (assumed independent)
    circuit = cirq.Circuit()
    circuit += state_circuit1
    circuit += state_circuit2_remapped

    # Prepare the ancilla
    circuit.append(cirq.H(ancilla))
    # Append a controlled-SWAP gate; in Cirq we can use:
    cswap = cirq.SWAP.controlled()
    circuit.append(cswap(ancilla, qubit1, qubit2_remapped))
    circuit.append(cirq.H(ancilla))
    circuit.append(cirq.measure(ancilla, key='anc'))

    # Print the swap-test circuit diagram
    print("Swap Test Circuit:")
    print(circuit)#.to_text_diagram()
    print("-" * 40)

    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=repetitions)
    counts = result.histogram(key='anc')
    p0 = counts.get(0, 0) / repetitions
    return p0

In [37]:
def verify_signature(b: int, signature, public_keys, c1: float, c2: float, swap_threshold=0.95, repetitions=100):
    """
    Performs signature verification using a quantum swap test.

    For each key i, the recipient:
      - Reconstructs the public state from the revealed key (using prepare_state_circuit with 1 copy).
      - Uses one copy of the stored public key (from public_keys) as the reference.
      - Performs a swap test between the two states.

    If the swap test yields an ancilla measurement of 0 with probability
    above the swap_threshold, the key is considered valid. Otherwise, an error is counted.

    Based on the total error_count compared to thresholds c1 and c2, the function returns:
      "1-ACC" if error_count ≤ c1 * M,
      "REJ" if error_count ≥ c2 * M,
      "0-ACC" otherwise.

    Parameters:
      b: message bit.
      signature: tuple (b, [k1^b, k2^b, ..., k_M^b]).
      public_keys: recipient's stored public key states for bit b (list of (circuit, qubits)).
      c1: acceptance threshold fraction.
      c2: rejection threshold fraction.
      swap_threshold: minimum swap test probability for a key to be considered a match.
      repetitions: number of repetitions for the swap test simulation.
    """
    M = len(public_keys[b])
    error_count = 0

    for i, (stored_circuit, stored_qubits) in enumerate(public_keys[b]):
        # Retrieve the revealed key for this public key.
        revealed_key = signature[1][i]
        # Reconstruct the state corresponding to the revealed key (1 copy).
        revealed_circuit, revealed_qubits = prepare_state_circuit(revealed_key, 1)
        # Use the first qubit of the stored public key as the reference. public key has T copies of same qubit print(stored_qubits)
        stored_qubit = stored_qubits[0]  # the circuit diagram has T qubits of storedqubits change 0 to any no in T-1


        # Run the swap test between the state prepared from the revealed key and the stored state.
        p0 = perform_swap_test(revealed_circuit, revealed_qubits[0], stored_circuit, stored_qubit, repetitions=repetitions)

        # If the swap test probability is below the threshold, count it as an error.
        if p0 < swap_threshold:
            error_count += 1

    # Uncomment the following line for debugging:
    print(f"Total swap-test errors: {error_count} out of {M}")

    if error_count <= c1 * M:
        return "1-ACC"  # Accepted and transferable.
    elif error_count >= c2 * M:
        return "REJ"    # Rejected.
    else:
        return "0-ACC"  # Uncertain validity.

# Example usage:
if __name__ == "__main__":
    # Set protocol parameters.
    M = 5     # Number of key pairs.
    L = 4     # Length of each bit-string.
    T = 3     # Number of copies per public key state (for signature verification, one copy is used).

    # Thresholds for verification.
    c1 = 0.2  # If errors are ≤20% of keys, signature is accepted as transferable.
    c2 = 0.5  # If errors are ≥50% of keys, signature is rejected.

    # Generate keys.
    private_keys, public_keys = generate_keys(M, L, T)

    # Alice signs a message bit b.
    b = 1  # For example, she signs the bit 1.
    signature = sign_message(b, private_keys)
    print("Signature generated:")
    print("Message bit:", signature[0])
    print("Revealed keys:", signature[1])

    # Verify the signature using the swap-test based method.
    result = verify_signature(b, signature, public_keys, c1, c2, swap_threshold=0.95, repetitions=100)
    print("\nVerification result (using swap test):", result)


Signature generated:
Message bit: 1
Revealed keys: ['1101', '1000', '0100', '0101', '1001']
Swap Test Circuit:
                                ┌──────────┐
0: ─────Ry(0.25π)─────────────────────────×───────────────────────────
                                          │
10: ────────────────Ry(0.25π)─────────────×───────────────────────────
                                          │
11: ─────────────────────────────Ry(0.25π)┼───────────────────────────
                                          │
12: ──────────────────────────────────────┼────Ry(0.25π)──────────────
                                          │
100: ───H─────────────────────────────────@────H───────────M('anc')───
                                └──────────┘
----------------------------------------
Swap Test Circuit:
                        ┌──────┐
0: ─────Ry(0)─────────────────×───────────────────────
                              │
10: ────────────Ry(0)─────────×───────────────────────
                              │
1

# **Key Distribution**

In [38]:
def distributed_key_distribution_test(public_key_info):
    """
    Simulates distributed key distribution using swap tests.

    For each public key (for a given message bit b), assume that
    Alice sends 4 copies (T = 4). Two copies go to Bob and two to Charlie.

    The test consists of:
      1. Bob performs a swap test on his two copies.
      2. Charlie performs a swap test on his two copies.
      3. Bob sends one copy to Charlie and they perform a swap test on the exchanged copies.

    public_key_info is a tuple (circuit, qubits) as generated by prepare_state_circuit.

    Returns True if all swap tests yield a probability close to 1 (here we use a threshold of 0.95).
    """
    # Unpack the public key state information
    circuit, qubits = public_key_info
    # In our distribution, we assume T = 4 copies.
    if len(qubits) < T:
        raise ValueError(f"Need at least {T}copies for distributed key distribution test.")

    # Split the 4 copies: Bob gets copies 0 and 1; Charlie gets copies 2 and 3.
    bob_copy1, bob_copy2 = qubits[0], qubits[1]
    charlie_copy1, charlie_copy2 = qubits[2], qubits[3]

    # Build state circuits for Bob and Charlie.
    # (Since the public key state is prepared by the same circuit,
    #  we assume the same state preparation.)
    state_circuit = circuit  # both parties have identical preparation circuits.

    # 1. Bob's local swap test
    p0_bob = perform_swap_test(state_circuit, bob_copy1, state_circuit, bob_copy2)

    # 2. Charlie's local swap test
    p0_charlie = perform_swap_test(state_circuit, charlie_copy1, state_circuit, charlie_copy2)

    # 3. Distributed swap test: Bob sends one copy (copy 1) to Charlie;
    #    perform swap test between Bob's copy 1 and Charlie's copy 1.
    p0_cross = perform_swap_test(state_circuit, bob_copy1, state_circuit, charlie_copy1)

    # Define a threshold (here we use 0.95 to account for simulation imperfections)
    threshold = 0.95
    if p0_bob >= threshold and p0_charlie >= threshold and p0_cross >= threshold:
        return True
    else:
        return False

# Example usage of key distribution test:
if __name__ == "__main__":
    # For key distribution, we choose T = 4 copies per public key.
    M = 3     # Number of key pairs (for demonstration)
    L = 4     # Length of each bit-string
    T = 4     # For key distribution, use 4 copies

    # Generate keys (we only need to test one message bit; here we choose bit 0)
    private_keys, public_keys = generate_keys(M, L, T)

    # Simulate key distribution tests for all public keys corresponding to message bit 0.
    all_tests_pass = True
    for i, public_key_info in enumerate(public_keys[0]):
        test_passed = distributed_key_distribution_test(public_key_info)
        print(f"Public key {i+1} distribution test passed: {test_passed}")
        if not test_passed:
            all_tests_pass = False

    if all_tests_pass:
        print("\nAll distributed swap tests passed. Recipients have identical public keys.")
    else:
        print("\nKey distribution test failed. Recipients may not have identical copies.")


Swap Test Circuit:
                                                                                ┌──────────┐
0: ─────Ry(0.25π)─────────────────────────────────────────────────────────────────────────×───────────────────────────
                                                                                          │
1: ─────────────────Ry(0.25π)─────────────────────────────────────────────────────────────┼───────────────────────────
                                                                                          │
2: ─────────────────────────────Ry(0.25π)─────────────────────────────────────────────────┼───────────────────────────
                                                                                          │
3: ─────────────────────────────────────────Ry(0.25π)─────────────────────────────────────┼───────────────────────────
                                                                                          │
10: ────────────────────────────────────────

# **Simulate Attack Scenarios**

In [39]:
def prepare_state_with_error(key: str, T: int, delta: float):
    """
    Prepares T copies of the state |f_k> = cos(j*theta)|0> + sin(j*theta)|1>
    with an error. One state is prepared with an extra rotation delta.
    """
    L = len(key)
    j = int(key, 2) % L
    theta = np.pi / (2 * L)

    qubits = [cirq.LineQubit(i) for i in range(T)]
    circuit = cirq.Circuit()

    # Apply an extra rotation delta to simulate error
    for q in qubits:
        circuit.append(cirq.ry(2 * j * theta + delta)(q))

    return circuit, qubits

# Example: Use the regular state preparation for the stored state,
# but simulate a swap test failure by using a different state (with delta != 0)
key = "1011"
T = 1
delta = 0.3  # nonzero delta to induce a state difference

# Correct state (stored state)
stored_circuit, stored_qubits = prepare_state_circuit(key, T)
# Erroneous state (revealed state with error)
erroneous_circuit, erroneous_qubits = prepare_state_with_error(key, T, delta)

# Perform swap test between the correct and erroneous states.
p0 = perform_swap_test(erroneous_circuit, erroneous_qubits[0], stored_circuit, stored_qubits[0], repetitions=1000)
print(f"Swap test probability (should be less than 1): {p0} hence error !")


Swap Test Circuit:
0: ─────Ry(0.845π)───────────────×──────────────────
                                 │
10: ─────────────────Ry(0.75π)───×──────────────────
                                 │
100: ───H────────────────────────@───H───M('anc')───
----------------------------------------
Swap test probability (should be less than 1): 0.988 hence error !


In [40]:
# --- Attack Scenario Simulations ---

def simulate_forging_attempt(M, L, T, c1, c2, swap_threshold=0.95, repetitions=100):
    """
    Simulates a forging attack where Eve forges a signature without knowing Alice's private keys.
    Here Eve generates a signature using random keys.
    """
    # Generate keys as Alice would.
    private_keys, public_keys = generate_keys(M, L, T)
    # Instead of using Alice's private keys, Eve generates random keys.
    forged_signature_keys = [''.join(random.choice(['0', '1']) for _ in range(L)) for _ in range(M)]
    forged_signature = (1, forged_signature_keys)  # assume message bit 1
    result = verify_signature(1, forged_signature, public_keys, c1, c2, swap_threshold, repetitions)
    print("Forging attempt verification result:", result)

def simulate_public_key_leakage(M, L, T_initial, T_leaked, c1, c2, swap_threshold=0.95, repetitions=100):
    """
    Simulates public key leakage by increasing the number of copies T available.
    The idea is that with too many copies, an attacker may obtain enough information
    to forge a signature. Here, we simulate this by regenerating keys with a higher T
    (which would allow for more accurate state tomography in practice).

    We then attempt to verify a forged signature (as in the forging scenario)
    and see if the security is weakened.

    Parameters:
      M: number of key pairs.
      L: length of each key (bit-string).
      T_initial: number of copies originally distributed.
      T_leaked: number of copies available to the attacker.
      c1: acceptance threshold fraction.
      c2: rejection threshold fraction.
      swap_threshold: threshold probability for swap test to consider a key valid.
      repetitions: number of repetitions in the swap test simulation.
    """
    # Generate keys with T_initial copies (as originally distributed).
    private_keys, public_keys = generate_keys(M, L, T_initial)

    # Now suppose an attacker obtains additional copies.
    # We simulate this by regenerating the public key states using the
    # original key strings but with T_leaked copies.
    leaked_public_keys = {0: [], 1: []}
    for b in [0, 1]:
        for i, (circ, qubits) in enumerate(public_keys[b]):
            # Retrieve the corresponding private key used to generate the public state.
            key_str = private_keys[i][b]
            new_circ, new_qubits = prepare_state_circuit(key_str, T_leaked)
            leaked_public_keys[b].append((new_circ, new_qubits))

    # Now, simulate a forging attempt:
    # A forged signature is created by choosing random L-bit strings for each key.
    forged_signature_keys = [''.join(random.choice(['0', '1']) for _ in range(L)) for _ in range(M)]
    forged_signature = (1, forged_signature_keys)

    # Verify the forged signature against the leaked public keys.
    result = verify_signature(1, forged_signature, leaked_public_keys, c1, c2, swap_threshold, repetitions)

    print("Public key leakage forging attempt verification result:", result)
    print("(Note: In a full protocol, leakage would allow state tomography and better forgery.)")

def simulate_quantum_state_tampering(M, L, T, c1, c2, delta, swap_threshold=0.95, repetitions=100):
    """
    Simulates quantum state tampering in the transmission channel.
    We simulate tampering by modifying the state (using an extra rotation delta)
    for the transmitted (revealed) public keys.
    """
    # Generate keys as normal.
    private_keys, public_keys = generate_keys(M, L, T)
    # Alice signs a message bit.
    b = 1
    signature = sign_message(b, private_keys)
    # Now, simulate tampering: for each key, instead of preparing the correct state,
    # the transmitted state is prepared with an error (delta).
    tampered_signature_keys = []
    for key in signature[1]:
        # Instead of using the correct key, simulate that the channel adds an error.
        # The attacker (or noise) applies an extra rotation delta.
        # We use prepare_state_with_error to simulate the tampered state.
        # For our simulation, we record the tampered key as the same key label,
        # but the verification will use the tampered state.
        tampered_signature_keys.append(key)  # key label remains the same
    tampered_signature = (b, tampered_signature_keys)

    # For verification, we modify the routine: when reconstructing the revealed state,
    # use the tampered preparation function.
    error_count = 0
    for i, (stored_circuit, stored_qubits) in enumerate(public_keys[b]):
        revealed_key = tampered_signature[1][i]
        # Instead of correct state preparation, prepare the state with error (delta).
        revealed_circuit, revealed_qubits = prepare_state_with_error(revealed_key, 1, delta)
        stored_qubit = stored_qubits[0]
        p0 = perform_swap_test(revealed_circuit, revealed_qubits[0], stored_circuit, stored_qubit, repetitions=repetitions)
        if p0 < swap_threshold:
            error_count += 1
    result = ("1-ACC" if error_count <= c1*M
              else "REJ" if error_count >= c2*M
              else "0-ACC")
    print("Quantum state tampering verification result:", result)
    print(f"(Tampering induced error in {error_count} out of {M} keys.)")

# --- Main simulation of attack scenarios ---

if __name__ == "__main__":
    # Protocol parameters
    M = 5      # number of key pairs
    L = 4      # bit-string length
    T = 3      # number of copies per public key state (for verification)
    c1 = 0.2   # acceptance threshold fraction
    c2 = 0.5   # rejection threshold fraction
    swap_threshold = 0.95
    repetitions = 100

    print("=== Forging Attempt Simulation ===")
    simulate_forging_attempt(M, L, T, c1, c2, swap_threshold, repetitions)

    print("\n=== Public Key Leakage Simulation ===")
    # For leakage, we simulate that many more copies are available.
    T_initial = 3
    T_leaked = 10  # More copies would allow better state estimation.
    simulate_public_key_leakage(M, L, T_initial, T_leaked, c1, c2, swap_threshold, repetitions)

    print("\n=== Quantum State Tampering Simulation ===")
    delta = 0.7  # A nonzero delta to simulate channel-induced error.
    simulate_quantum_state_tampering(M, L, T, c1, c2, delta, swap_threshold, repetitions)

=== Forging Attempt Simulation ===
Swap Test Circuit:
                                ┌──────────┐
0: ─────Ry(0.25π)─────────────────────────×───────────────────────────
                                          │
10: ────────────────Ry(0.75π)─────────────×───────────────────────────
                                          │
11: ─────────────────────────────Ry(0.75π)┼───────────────────────────
                                          │
12: ──────────────────────────────────────┼────Ry(0.75π)──────────────
                                          │
100: ───H─────────────────────────────────@────H───────────M('anc')───
                                └──────────┘
----------------------------------------
Swap Test Circuit:
                                ┌──────────┐
0: ─────Ry(0.75π)─────────────────────────×───────────────────────────
                                          │
10: ────────────────Ry(0.75π)─────────────×───────────────────────────
                                  

# **Optimize Parameters**

In [41]:
def optimize_parameters(M_values, T_values, c1_values, c2_values, L, swap_threshold=0.95, repetitions=100):
    """
    For a set of parameter values:
      - M: number of key pairs
      - T: number of copies per public key state
      - c1: acceptance threshold fraction (if errors ≤ c1*M, signature is accepted)
      - c2: rejection threshold fraction (if errors ≥ c2*M, signature is rejected)
    and fixed key length L,
    this function generates keys and then simulates a forging attempt by
    generating random keys. It then verifies the forged signature using the
    swap-test-based verification routine. The idea is that, as M increases,
    the probability that a forged signature passes should become exponentially small.

    The function prints the results for each parameter set.
    """
    results = []
    for M in M_values:
        for T in T_values:
            for c1 in c1_values:
                for c2 in c2_values:
                    if c1 >= c2:
                        continue  # thresholds must satisfy c1 < c2.

                    # Generate keys with T copies.
                    private_keys, public_keys = generate_keys(M, L, T)

                    # Forge a signature: choose random L-bit strings for each key.
                    forged_signature_keys = [''.join(random.choice(['0', '1']) for _ in range(L)) for _ in range(M)]
                    forged_signature = (1, forged_signature_keys)

                    # Verify the forged signature.
                    result = verify_signature(1, forged_signature, public_keys, c1, c2, swap_threshold, repetitions)

                    results.append((M, T, c1, c2, result))
                    print(f"M={M}, T={T}, c1={c1}, c2={c2} -> Verification Result: {result}")
    return results

# Example usage:
if __name__ == "__main__":
    print("\n=== Parameter Optimization Simulation ===")

    # Define parameter ranges.
    M_values = [5, 10, 15, 20]       # Increasing M should exponentially lower forgery probability.
    T_values = [3, 4]                # Number of copies per public key.
    c1_values = [0.1, 0.2]           # Lower acceptance threshold.
    c2_values = [0.4, 0.5]           # Higher rejection threshold.
    L = 4                          # Length of each key (bit-string).

    # Run the optimization simulation.
    r = optimize_parameters(M_values, T_values, c1_values, c2_values, L, swap_threshold=0.95, repetitions=100)
    print("\n=== Accepted Results Only ===")
    for M, T, c1, c2, result in r:
        if "ACC" in result:
            print(f"M={M}, T={T}, c1={c1}, c2={c2} -> Verification Result: {result}")


Streaming output truncated to the last 5000 lines.
----------------------------------------
Swap Test Circuit:
                                ┌──────────┐
0: ─────Ry(0.75π)─────────────────────────×───────────────────────────
                                          │
10: ────────────────Ry(0.25π)─────────────×───────────────────────────
                                          │
11: ─────────────────────────────Ry(0.25π)┼───────────────────────────
                                          │
12: ──────────────────────────────────────┼────Ry(0.25π)──────────────
                                          │
100: ───H─────────────────────────────────@────H───────────M('anc')───
                                └──────────┘
----------------------------------------
Swap Test Circuit:
                        ┌──────┐
0: ─────Ry(0)─────────────────×───────────────────────
                              │
10: ────────────Ry(0)─────────×───────────────────────
                              │
1

# **Performance Analysis**

In [42]:
def verify_signature_with_errors(b: int, signature, public_keys, c1: float, c2: float, swap_threshold=0.95, repetitions=100, error_probability=0.0):
    """
    Verifies the signature using a swap test on each key and counts the number of errors.
    For each key i, the recipient:
      - Reconstructs the state from the revealed key (using prepare_state_circuit with 1 copy).
      - Uses one copy of the stored public key as reference.
      - Runs a swap test between the two states.
    With probability error_probability, we simulate an additional failure.

    Returns a tuple (outcome, error_count) where outcome is one of:
       "1-ACC" if error_count ≤ c1 * M,
       "REJ" if error_count ≥ c2 * M,
       "0-ACC" otherwise.
    """
    M = len(public_keys[b])
    error_count = 0

    for i, (stored_circuit, stored_qubits) in enumerate(public_keys[b]):
        # Retrieve the revealed key from the signature.
        revealed_key = signature[1][i]
        # Reconstruct the state corresponding to the revealed key (1 copy).
        revealed_circuit, revealed_qubits = prepare_state_circuit(revealed_key, 1)
        # Use the first qubit of the stored public key as reference.
        stored_qubit = stored_qubits[0]

        # Run the swap test between the revealed and stored state.
        p0 = perform_swap_test(revealed_circuit, revealed_qubits[0],
                               stored_circuit, stored_qubit, repetitions=repetitions)

        # Introduce additional simulated noise:
        if random.random() < error_probability:
            # Simulate that the swap test fails for this key.
            p0 = 0.0

        # Count an error if the swap test probability is below the threshold.
        if p0 < swap_threshold:
            error_count += 1

    if error_count <= c1 * M:
        outcome = "1-ACC"
    elif error_count >= c2 * M:
        outcome = "REJ"
    else:
        outcome = "0-ACC"

    return outcome, error_count

def simulate_multiple_recipients_verification(M, L, T, num_recipients, c1, c2, swap_threshold=0.95, repetitions=100, error_probability=0.0):
    """
    Simulates signature verification across multiple recipients.
    All recipients share the same public keys. Alice signs a message (here we choose bit 1).

    For each recipient, the verification is performed (with an optional error_probability
    to simulate noise). The function prints and returns a list of (outcome, error_count) for each recipient.

    Also prints the average error count over all recipients.
    """
    # Generate keys (and corresponding public keys) with T copies.
    private_keys, public_keys = generate_keys(M, L, T)
    # Alice signs a message bit b (here we choose b = 1 for example).
    b = 1
    signature = sign_message(b, private_keys)

    results = []
    for r in range(num_recipients):
        outcome, err_count = verify_signature_with_errors(b, signature, public_keys, c1, c2, swap_threshold, repetitions, error_probability)
        results.append((outcome, err_count))

    avg_error = sum(err for (outcome, err) in results) / num_recipients
    print(f"Average error count over {num_recipients} recipients: {avg_error} (out of {M})")
    for idx, (outcome, err) in enumerate(results):
        print(f"Recipient {idx+1}: Outcome = {outcome}, Errors = {err}")
    return results

def simulate_scalability():
    """
    Runs performance analysis by measuring error rates for signature verification
    while increasing the number of recipients.
    """
    # Fixed parameters.
    M = 10         # Number of key pairs.
    L = 4          # Key length (bit-string).
    T = 3          # Copies per public key.
    c1 = 0.2       # Acceptance threshold (errors ≤ 20% of keys => valid).
    c2 = 0.5       # Rejection threshold (errors ≥ 50% of keys => invalid).
    swap_threshold = 0.95
    repetitions = 100
    error_probability = 0.05  # Simulate a 5% chance of extra error per key.

    # Test scalability with different numbers of recipients.
    for num_recipients in [1, 5, 10, 20]:
        print(f"\n--- Simulation for {num_recipients} recipient(s) ---")
        simulate_multiple_recipients_verification(M, L, T, num_recipients, c1, c2, swap_threshold, repetitions, error_probability)

# Example usage:
if __name__ == "__main__":
    print("\n=== Performance Analysis: Verification Error Rates and Scalability ===")
    simulate_scalability()



=== Performance Analysis: Verification Error Rates and Scalability ===

--- Simulation for 1 recipient(s) ---
Swap Test Circuit:
                              ┌─────────┐
0: ─────Ry(0.5π)───────────────────────×──────────────────────────
                                       │
10: ───────────────Ry(0.5π)────────────×──────────────────────────
                                       │
11: ───────────────────────────Ry(0.5π)┼──────────────────────────
                                       │
12: ───────────────────────────────────┼────Ry(0.5π)──────────────
                                       │
100: ───H──────────────────────────────@────H──────────M('anc')───
                              └─────────┘
----------------------------------------
Swap Test Circuit:
                              ┌─────────┐
0: ─────Ry(0.5π)───────────────────────×──────────────────────────
                                       │
10: ───────────────Ry(0.5π)────────────×──────────────────────────
          

# **Extend to Multi-Bit Messages**

In [43]:
def encode_message(message: str, r: int) -> str:
    """
    Encodes a binary message using a repetition code.
    Each bit is repeated r times.
    For example, encode_message("101", 3) -> "111000111".
    """
    return "".join(bit * r for bit in message)

def decode_message(encoded_message: str, r: int) -> str:
    """
    Decodes a repetition-encoded binary message by majority vote.
    Assumes the length of encoded_message is a multiple of r.
    """
    m = len(encoded_message) // r
    decoded = ""
    for i in range(m):
        block = encoded_message[i*r:(i+1)*r]
        decoded += "1" if block.count("1") > block.count("0") else "0"
    return decoded

def sign_multi_bit_message(message: str, r: int, private_keys_encoded: list) -> tuple:
    """
    Given an m-bit message (as a string) and a repetition factor r,
    encodes the message using a repetition code and then signs each encoded bit.

    private_keys_encoded is a list of key pairs (one per encoded bit)
    and its length must equal len(message) * r.

    Returns a tuple (encoded_message, signature_keys), where:
      - encoded_message is the repetition-encoded message,
      - signature_keys is a list of revealed keys (one per encoded bit).
    """
    encoded_message = encode_message(message, r)
    signature_keys = [key_pair[int(bit)] for key_pair, bit in zip(private_keys_encoded, encoded_message)]
    return (encoded_message, signature_keys)

def verify_encoded_bit(b: int, signature_key: str, public_key: tuple, swap_threshold: float,
                       repetitions: int, error_probability: float) -> int:
    """
    Verifies a single encoded bit using a swap test.

    b: expected bit value (0 or 1) for this encoded bit.
    signature_key: the revealed key (a bit-string) for this encoded bit.
    public_key: a tuple (circuit, qubits) for the stored public key.

    The function reconstructs the state from the revealed key (using 1 copy)
    and runs a swap test against the stored state (using its first qubit).

    With additional probability error_probability we simulate an error.
    If the swap test probability (p0) is below swap_threshold,
    we flip the bit.

    Returns the verified bit (0 or 1).
    """
    revealed_circuit, revealed_qubits = prepare_state_circuit(signature_key, 1)
    stored_circuit, stored_qubits = public_key
    p0 = perform_swap_test(revealed_circuit, revealed_qubits[0],
                           stored_circuit, stored_qubits[0],
                           repetitions=repetitions)
    if random.random() < error_probability:
        p0 = 0.0  # Simulate an error.
    return b if p0 >= swap_threshold else 1 - b

def verify_multi_bit_signature(encoded_signature: tuple, public_keys_encoded: dict,
                               swap_threshold: float, repetitions: int, error_probability: float,
                               r: int) -> tuple:
    """
    Verifies a multi-bit (repetition-encoded) signature.

    encoded_signature: tuple (encoded_message, signature_keys) as produced by sign_multi_bit_message.
    public_keys_encoded: dictionary with keys 0 and 1. Each maps to a list of stored public keys (tuples)
                         corresponding to that bit.

    For each encoded bit, the appropriate stored public key is selected
    based on the expected bit value.

    Returns a tuple (decoded_message, verified_encoded_message) where:
       - verified_encoded_message is the string of verified encoded bits (length = m*r)
       - decoded_message is the final decoded message (length = m).
    """
    encoded_message, signature_keys = encoded_signature
    verified_bits = ""
    for i, (bit_char, sk) in enumerate(zip(encoded_message, signature_keys)):
        b = int(bit_char)
        # Retrieve the stored public key corresponding to the expected bit value.
        pk = public_keys_encoded[b][i]
        verified_bit = verify_encoded_bit(b, sk, pk, swap_threshold, repetitions, error_probability)
        verified_bits += str(verified_bit)
    decoded = decode_message(verified_bits, r)
    return decoded, verified_bits

def simulate_multi_bit_signing_and_verification(message: str, r: int, M_total: int,
                                                  L: int, T: int, swap_threshold: float,
                                                  repetitions: int, error_probability: float):
    """
    Simulates multi-bit signing and verification.

    message: the original m-bit message (as a string, e.g., "1010").
    r: repetition factor (each bit is repeated r times).
    M_total: total number of key pairs required (should equal len(message) * r).
    L: key length.
    T: number of copies per public key state.
    swap_threshold, repetitions, error_probability: parameters for swap test simulation.

    The function:
      - Generates keys for each encoded bit,
      - Signs the encoded message,
      - Verifies each encoded bit via swap tests,
      - Decodes the final message.

    It prints the original message, encoded message, verified encoded message, and decoded message.
    """
    m = len(message)
    if M_total != m * r:
        raise ValueError("M_total must equal len(message) * r.")

    # Generate keys for each encoded bit.
    private_keys_encoded, public_keys_encoded = generate_keys(M_total, L, T)
    # public_keys_encoded is a dictionary with keys 0 and 1.

    # Alice signs the message.
    encoded_signature = sign_multi_bit_message(message, r, private_keys_encoded)
    print("Original message:", message)
    print("Encoded message:", encoded_signature[0])

    # A recipient verifies the signature.
    decoded_message, verified_encoded_message = verify_multi_bit_signature(
        encoded_signature, public_keys_encoded, swap_threshold, repetitions, error_probability, r
    )
    print("Verified encoded message:", verified_encoded_message)
    print("Decoded message:", decoded_message)
    return decoded_message, verified_encoded_message

# Example usage:
if __name__ == "__main__":
    print("\n=== Multi-Bit Signing and Verification Simulation ===")
    # Original message (m bits)
    message = "1010"         # For example, a 4-bit message.
    r = 3                    # Repetition factor: each bit is repeated 3 times.
    M_total = len(message) * r   # Total number of key pairs.
    L = 4                    # Length of each key (bit-string).
    T = 3                    # Number of copies per public key state.

    # Swap test and verification parameters.
    swap_threshold = 0.95
    repetitions = 100
    error_probability = 0.05   # Simulate a 5% extra error chance per key.

    simulate_multi_bit_signing_and_verification(message, r, M_total, L, T, swap_threshold, repetitions, error_probability)



=== Multi-Bit Signing and Verification Simulation ===
Original message: 1010
Encoded message: 111000111000
Swap Test Circuit:
                        ┌──────┐
0: ─────Ry(0)─────────────────×───────────────────────
                              │
10: ────────────Ry(0)─────────×───────────────────────
                              │
11: ─────────────────────Ry(0)┼───────────────────────
                              │
12: ──────────────────────────┼────Ry(0)──────────────
                              │
100: ───H─────────────────────@────H───────M('anc')───
                        └──────┘
----------------------------------------
Swap Test Circuit:
                              ┌─────────┐
0: ─────Ry(0.5π)───────────────────────×──────────────────────────
                                       │
10: ───────────────Ry(0.5π)────────────×──────────────────────────
                                       │
11: ───────────────────────────Ry(0.5π)┼──────────────────────────
                   

In [44]:
# def verify_signature_classical(b: int, signature, public_keys, c1: float, c2: float, error_probability=0.0):
#     """
#     Simulates the signature verification.

#     For each key i, the recipient regenerates the public state using
#     the revealed key from the signature and compares it with their copy.

#     We simulate the quantum verification by using a classical equality test.
#     In an honest run (with error_probability=0), all keys will match.
#     To simulate noise (or cheating), you may set error_probability > 0.

#     Parameters:
#       b: message bit.
#       signature: tuple (b, [k1^b, k2^b, ..., k_M^b]).
#       public_keys: recipient's copy of public key states for bit b,
#                    as generated earlier.
#       c1: acceptance threshold fraction (if errors ≤ c1*M, accept as transferable).
#       c2: rejection threshold fraction (if errors ≥ c2*M, reject).
#       error_probability: probability with which an error occurs in verifying a key.
#                          (Default is 0, i.e. no error.)

#     Returns:
#       A string indicating the verification result:
#         "1-ACC" for valid & transferable,
#         "0-ACC" for valid but uncertain transferability,
#         "REJ" for rejected.
#     """
#     M = len(public_keys[b])
#     error_count = 0

#     for i, (circuit, qubits) in enumerate(public_keys[b]):
#         revealed_key = signature[1][i]
#         # In a real scenario, the recipient would perform a quantum verification (e.g., swap test)
#         # to check that the state prepared by revealed_key matches the stored public key.
#         # For simulation, if the revealed key matches the key used during generation, verification passes.
#         # Here we simulate noise: with probability error_probability, we count a verification error.
#         if random.random() < error_probability:
#             error_count += 1
#         # Otherwise, if honest, the revealed key is assumed correct.

#     # Uncomment the following line to print the error count during debugging:
#     # print(f"Total errors: {error_count} out of {M}")

#     if error_count <= c1 * M:
#         return "1-ACC"  # Accepted, transferable.
#     elif error_count >= c2 * M:
#         return "REJ"    # Rejected.
#     else:
#         return "0-ACC"  # Uncertain validity.

# # Example usage:
# if __name__ == "__main__":
#     # Set protocol parameters.
#     M = 5     # Number of key pairs.
#     L = 4     # Length of each bit-string.
#     T = 3     # Number of copies per public key state.

#     # Thresholds: you may adjust these to simulate different outcomes.
#     c1 = 0.2  # Acceptance threshold (if errors are at most 20% of keys, signature is accepted).
#     c2 = 0.5  # Rejection threshold (if errors are at least 50% of keys, signature is rejected).

#     # Generate keys.
#     private_keys, public_keys = generate_keys(M, L, T)

#     # Alice signs a message bit b.
#     b = 1  # For example, she signs the bit 1.
#     signature = sign_message(b, private_keys)
#     print("Signature generated:")
#     print("Message bit:", signature[0])
#     print("Revealed keys:", signature[1])

#     # Scenario 1: Honest signature (error_probability = 0).
#     # result_honest = verify_signature(b, signature, public_keys, c1, c2, error_probability=0.0)
#     # print("\nVerification result (honest signature):", result_honest)

#     # Scenario 2: Slight noise, uncertain validity.
#     # Uncomment the following lines to simulate a scenario where errors occur
#     # with a moderate probability (e.g., 30% chance per key) leading to "0-ACC".
#     # result_uncertain = verify_signature(b, signature, public_keys, c1, c2, error_probability=0.6)
#     # print("\nVerification result (uncertain validity with noise):", result_uncertain)

#     # Scenario 3: High noise, leading to rejection.
#     # Uncomment the following lines to simulate a scenario where errors occur
#     # with high probability (e.g., 60% chance per key) leading to "REJ".
#     result_reject = verify_signature_classical(b, signature, public_keys, c1, c2, error_probability=0.9)
#     print("\nVerification result (rejected due to high errors):", result_reject)